In [1]:
!pip install marshmallow-jsonapi

    100% |████████████████████████████████| 51kB 8.5MB/s ta 0:00:01


## Generate URL

In [1]:
params = {
    "filter": "nelements<3",
    "response_format": "json",
    "email_address": "dwinston@lbl.gov",
    "response_limit": "10",
    "response_fields": "id,nelements,chemical_formula",
    "sort": "-nelements",
}

In [2]:
def concatinator(endpoint, params):
    return f"{endpoint}?{urlencode(params)}"

In [3]:
from urllib.parse import urlparse, quote_plus, urlencode, parse_qs
endpoint = "https://materialsproject.org/optimade/0.9.6/structures"
url = concatinator(endpoint, params)
print(url)

https://materialsproject.org/optimade/0.9.6/structures?filter=nelements%3C3&response_format=json&email_address=dwinston%40lbl.gov&response_limit=10&response_fields=id%2Cnelements%2Cchemical_formula&sort=-nelements


## Parse input URL

In [6]:
from urllib.parse import urlparse,quote_plus,parse_qs
import pymongo
def parseURL(url):
    """
    @param url: input url
    @return 1. path -- the path of user's request, ex: /optimade/0.9.6/structures
            2. query -- user's query fields which is a dictionary with keys filter, 
                        response_format, email_address, response limit, response_fields,
                        and sort
    """
    parsed = urlparse(url)
    path = parsed.path
    query = parse_qs(parsed.query)
    
    query['query'] = "filter = %s" % query['filter'][0]
    query.pop('filter',None)
    query['response_limit'] = query['response_limit'][0]
    query['response_fields'] = query['response_fields'][0].split(",")
    json_sort_query = query['sort'][0]
    # change from json sorting format to mongoDB sorting method
    if(json_sort_query.find("-") > -1):
        query['sort'] = (json_sort_query[json_sort_query.find("-")+1:], pymongo.ASCENDING)
    else:
        query['sort'] = (json_sort_query, pymongo.DESCENDING)
    splitted = path.split("/")
    #adding path params to the result
    query["endpoint"] = splitted[1]
    query["grammar"] = splitted[2]
    query["entry"] = splitted[3]
    return query

In [7]:
result = parseURL(url)
result

{'response_format': ['json'],
 'email_address': ['dwinston@lbl.gov'],
 'response_limit': '10',
 'response_fields': ['id', 'nelements', 'chemical_formula'],
 'sort': ('nelements', 1),
 'query': 'filter = nelements<3',
 'endpoint': 'optimade',
 'grammar': '0.9.6',
 'entry': 'structures'}

In [72]:
import ast
def mongoDBQueryBuilder(query, mongoCollection):
    q = query['query']
    contents = !mongoconverter {q}
    return contents

In [73]:
import pymongo
from pymongo import MongoClient
client = MongoClient()
db=client.test_database
test_collection = db.test_collection
print(result)
mongoDBQueryBuilder(result,test_collection)

{'response_format': ['json'], 'email_address': ['dwinston@lbl.gov'], 'response_limit': '10', 'response_fields': ['id', 'nelements', 'chemical_formula'], 'sort': ('nelements', 1), 'query': 'filter = nelements<3', 'endpoint': 'optimade', 'grammar': '0.9.6', 'entry': 'structures'}


['/bin/bash: 3: No such file or directory']

## Get Data

In [32]:
import pymongo
from pymongo import MongoClient
client = MongoClient()
db=client.test_database
test_collection = db.test_collection
cursor = test_collection.find({'nelements': {'$gte': 2.0}})

In [33]:
for i in cursor:
    print(i)

{'_id': ObjectId('5c577f2026ef0c4c2bf9326a'), 'elements': ['O', 'Ti'], 'nelements': 2, 'pretty_formula': 'TiO2', 'formula_anonymous': 'AB2', 'material_id': 'mp-775938'}
{'_id': ObjectId('5c577f2026ef0c4c2bf9326b'), 'elements': ['O', 'Ti'], 'nelements': 2, 'pretty_formula': 'TiO2', 'formula_anonymous': 'AB2', 'material_id': 'mp-766454'}
{'_id': ObjectId('5c577f2026ef0c4c2bf9326c'), 'elements': ['O', 'Ti'], 'nelements': 2, 'pretty_formula': 'TiO2', 'formula_anonymous': 'AB2', 'material_id': 'mp-9173'}
{'_id': ObjectId('5c577f2026ef0c4c2bf9326d'), 'elements': ['O', 'Ti'], 'nelements': 2, 'pretty_formula': 'Ti6O', 'formula_anonymous': 'AB6', 'material_id': 'mp-882'}
{'_id': ObjectId('5c577f2026ef0c4c2bf9326e'), 'elements': ['O', 'Ti'], 'nelements': 2, 'pretty_formula': 'TiO2', 'formula_anonymous': 'AB2', 'material_id': 'mvc-13391'}
{'_id': ObjectId('5c577f2026ef0c4c2bf9326f'), 'elements': ['O', 'Ti'], 'nelements': 2, 'pretty_formula': 'TiO2', 'formula_anonymous': 'AB2', 'material_id': 'mvc

# After Getting MongoDB result back

## Build model for compound

In [78]:
from marshmallow_jsonapi import Schema, fields
class Compound():
    def __init__(self, elements, nelements, pretty_formula, formula_anonymous, material_id):
        self.elements = elements
        self.nelements = nelements
        self.pretty_formula = pretty_formula
        self.formula_anonymous = formula_anonymous
        self.material_id = material_id
        self.id = material_id 

class CompoundSchema(Schema):
#     chemsys = fields.Str()
    elements = fields.List(fields.String)
    nelements = fields.Int()
    pretty_formula = fields.Str()
    formula_anonymous = fields.Str()
    material_id = fields.Str()
    id = fields.Str()
    
    class Meta:
        type_ = "compound"
        strict = True

compound_schema = CompoundSchema()

In [79]:
import pymongo
from pymongo import MongoClient
client = MongoClient()
db=client.test_database
test_collection = db.test_collection
cursor = test_collection.find()
counter = 0
data = []
for document in cursor:
    d = CompoundSchema().dump( Compound( 
                                    document["elements"], 
                                    document["nelements"], 
                                    document["pretty_formula"], 
                                    document["formula_anonymous"],
                                    document["material_id"]))
    data.append(d.data)

In [80]:
len(data)

87

In [81]:
data

[{'data': {'type': 'compound',
   'attributes': {'pretty_formula': 'TiO2',
    'elements': ['O', 'Ti'],
    'nelements': 2,
    'material_id': 'mp-775938',
    'formula_anonymous': 'AB2'},
   'id': 'mp-775938'}},
 {'data': {'type': 'compound',
   'attributes': {'pretty_formula': 'TiO2',
    'elements': ['O', 'Ti'],
    'nelements': 2,
    'material_id': 'mp-766454',
    'formula_anonymous': 'AB2'},
   'id': 'mp-766454'}},
 {'data': {'type': 'compound',
   'attributes': {'pretty_formula': 'TiO2',
    'elements': ['O', 'Ti'],
    'nelements': 2,
    'material_id': 'mp-9173',
    'formula_anonymous': 'AB2'},
   'id': 'mp-9173'}},
 {'data': {'type': 'compound',
   'attributes': {'pretty_formula': 'Ti6O',
    'elements': ['O', 'Ti'],
    'nelements': 2,
    'material_id': 'mp-882',
    'formula_anonymous': 'AB6'},
   'id': 'mp-882'}},
 {'data': {'type': 'compound',
   'attributes': {'pretty_formula': 'TiO2',
    'elements': ['O', 'Ti'],
    'nelements': 2,
    'material_id': 'mvc-13391',
  

In [2]:
from marshmallow import Schema, fields, pprint
import os, subprocess
from urllib.parse import urlparse,quote_plus,parse_qs
import pymongo
from pymongo import MongoClient
from urllib.parse import urlparse, quote_plus, urlencode, parse_qs
import ast
import datetime
from models_schema import *

def parseURL(url, alias):
    """
    @param url: input url
    @return 1. path -- the path of user's request, ex: /optimade/0.9.6/structures
            2. query -- user's query fields which is a dictionary with keys filter,
                        response_format, email_address, response limit, response_fields,
                        and sort
    """
    parsed = urlparse(url)
    path = parsed.path
    query = parse_qs(parsed.query)

    result = {}
    result['response_limit'] = int(query['response_limit'][0])
    result['response_fields'] = query['response_fields'][0].split(",")
    # swap content in query['response_fields'] to format in alias
    for i in range(len(query['response_fields'])):
        if(query['response_fields'][i] in alias):
            result['response_fields'][i] = alias[query['response_fields'][i]]

    json_sort_query = query['sort'][0]
    # change from json sorting format to mongoDB sorting method, TODO: change to support multiple sort objects
    if(json_sort_query.find("-") > -1):
        result['sort'] = (json_sort_query[json_sort_query.find("-")+1:], pymongo.ASCENDING)
    else:
        quresultery['sort'] = (json_sort_query, pymongo.DESCENDING)
    splitted = path.split("/")
    #adding path params to the result
    result["endpoint"] = splitted[1]
    result["api_version"] = splitted[2] # assuming that our grammar is our api version
    result["entry"] = splitted[3]

    result['query'] = "filter = %s" % query['filter'][0]
    # result.pop('filter',None)

    out = subprocess.Popen(['mongoconverter', result['query']],
               stdout=subprocess.PIPE,
               stderr=subprocess.STDOUT)
    stdout,stderr = out.communicate()

    # do i need to do try/except and throw custom error here?
    result['query'] = ast.literal_eval(stdout.decode("ascii"))

    return result

def generateSampleURL(endpoint, params):
    return f"{endpoint}?{urlencode(params)}"

def getDataFromDb(query):


    client = MongoClient()
    db=client.test_database
    test_collection = db.test_collection
    # print(result['sort'])
    cursor = test_collection.find(query['query'], projection=query['response_fields'])\
                            .sort([query['sort']])\
                            .limit(query['response_limit'])
    return cursor
#### START SAMPLE DATA ####
alias = {
    "chemical_formula":"formula_anonymous",
    "formula_prototype": "pretty_formula",
}
endpoint = "https://materialsproject.org/optimade/0.9.6/structures"
params = {
    "filter": "nelements<3",
    "response_format": "json",
    "email_address": "dwinston@lbl.gov",
    "response_limit": "10",
    "response_fields": "id,nelements,material_id,elements,formula_prototype",
    "sort": "-nelements",
}
#### END SAMPLE DATA ####

#### START OF PARSING URL ####
url = generateSampleURL(endpoint, params)
query = parseURL(url, alias)
#### END OF PARSING URL ####

# get data from database
cursor = getDataFromDb(query)

# organizing data into an array
data = []
data_returned_counter = 0
for document in cursor:
    structure_schema = StructureSchema().dump(Structure(param=document))
    data.append(structure_schema.data)
    data_returned_counter += 1

# generating other information needed in the response per specification
links = Links(None, None)
meta = Meta(
            query = {"representation":"/structures/?filter=a=1 AND b=2"},
            api_version = query.get("api_version"),
            time_stamp = datetime.datetime.utcnow().isoformat(),
            data_returned = data_returned_counter,
            more_data_available = True,
            )
data = Data(data)
links_schema = LinksSchema().dump(links)
meta_schema = MetaSchema().dump(meta)
data_schema = DataSchema().dump(data)
response_schema = Response(links_schema, meta_schema, data_schema)

response_schema